In [1]:
# Work 11
## Merging all prepared variables together
## Numerical data transformation: distribution check, normalization, standardization
## Categorical data transformation: one hot encoding

In [2]:
# loading libraries
import torch
torch.cuda.empty_cache() 
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image
import pandas as pd
import pyproj
from pathlib import Path
import sklearn
import seaborn as sn
from scipy import stats
from tabulate import tabulate
import statsmodels.api as sm
import statsmodels.tools as tools
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
import category_encoders as ce
from sklearn import svm
import mlxtend
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# loading written functions
from mtp_function_yl import *

In [11]:
# version control
%load_ext watermark
%watermark  -u -d -v -m --iversions

Last updated: 2024-11-22

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.14.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.0-124-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 48
Architecture: 64bit

cv2              : 4.8.1
pandas           : 2.0.3
seaborn          : 0.12.2
mlxtend          : 0.23.1
scipy            : 1.14.0
matplotlib       : 3.7.2
shapely          : 2.0.2
numpy            : 1.26.4
PIL              : 10.0.0
sklearn          : 1.3.0
pyproj           : 3.6.1
category_encoders: 2.6.3
statsmodels      : 0.14.4
torch            : 2.0.1



In [4]:
# loading prepared variable files 
# for accident points
# curb related variables and entropy variables
dfac_cr = pd.read_csv('/home/yelhe/script/mt/output/accident_crevall_new.csv')
# traffic transport infrastructure variables
dfac_tt = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_final.csv')

# for random pseudo points
# curb related variables and entropy variables
dfrp_cr = pd.read_csv('/home/yelhe/script/mt/output/pseudo_crevall_new.csv')
# traffic transport infrastructure variables
dfrp_tt = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_rp_final.csv')

# merging variables for accident locations
dfac_cr = dfac_cr.rename(columns = {'id':'Nr'})
dfac = dfac_cr.merge(dfac_tt, how = 'left', on = ['Nr'])
# print(dfac.isnull().values.any())
# merging variables for random pseudo locations
dfrp = dfrp_cr.merge(dfrp_tt, how = 'left', on = ['id'])

# saving into files
# raw variables of accident points
dfac.to_csv('/home/yelhe/script/mtp/redo_model_input/acc_rawvar.csv', index = False)
# raw variables of random pseudo points
dfrp.to_csv('/home/yelhe/script/mtp/redo_model_input/pse_rawvar.csv', index = False)

In [6]:
# dropping unnecessary variables
dfacr = dfac.drop(columns = ['z_stznr', 'r_surface'])
dfrpr = dfrp.drop(columns = ['z_stznr', 'r_surface'])
# generating variable: acp, 1 for presence and 0 for absence, to represent presence of accidents
dfrpr = dfrpr.rename(columns = {'id':'Nr'})
dfacr = dfacr.assign(acp = 1)
dfrpr = dfrpr.assign(acp = 0)
# combining these two datasets
df_acp_raw = pd.concat([dfacr, dfrpr], ignore_index = True, keys = ['ac','rp'])
# saving to file
df_acp_raw.to_csv('/home/yelhe/script/mtp/redo_model_input/model_acp_rawvar.csv', index = False)

In [ ]:
# For numeric data transformation

In [25]:
# loading above-processed data
acpr = pd.read_csv('/home/yelhe/script/mtp/redo_model_input/model_acp_rawvar.csv')
# dropping unnecessary columns
acpr = acpr.drop(columns = ['d_carcount', 'd_bicyclecount', 'dgvm', 'dcurb', 'curbtype'])
# creating list of variable names
# numeric variables
c1 = ['cmean', 'cmin', 'csum', 'cmax', 'cp', 'mewmin', 'megmin', 'iemin',
      'mewmax', 'megmax', 'iemax', 'mew', 'meg', 'ie',  'dbusl', 'dtraml',
      'dtrainl', 'dplight', 'dstation', 'dparkcar', 'dparktw', 'dpedcro',
      'dstopsign', 'dtrafficarea', 'droad','droad_sl','dvfpath',
      'gvm_dwv', 'gvm_msp', 'gvm_asp', 'r_width', 'speedlimit',  
      'bicyclecount', 'carcount']
# categorical variables
c2 = ['z_qnr', 'z_knr','trafficarea','r_width','speedlimit']
# all variables
c3 = ['Nr', 'cmean', 'cmin', 'csum', 'cmax', 'cp', 'mewmin', 'megmin', 'iemin',
      'mewmax', 'megmax', 'iemax', 'mew', 'meg', 'ie',  'dbusl', 'dtraml',
      'dtrainl', 'dplight', 'dstation', 'dparkcar', 'dparktw', 'dpedcro',
      'dstopsign', 'dtrafficarea', 'droad','droad_sl','dvfpath',
      'gvm_dwv', 'gvm_msp', 'gvm_asp', 'r_width', 'speedlimit',  
      'bicyclecount', 'carcount','z_qnr', 'z_knr','trafficarea',
      'r_width','speedlimit']
# 'Nr' + c1
c4 = c1.copy()
c4.insert(0, 'Nr')
# 'Nr' + c2
c5 = c2.copy()
c5.insert(0, 'Nr')

In [ ]:
# normalization
# normaltest pvalue (if it's bigger than alpha for example 0.05, we could say it's normal)
nmpv = []
for i in range(0, len(c1)):
    nmpv.append((stats.normaltest(acpr[c1[i]])).pvalue)
dfnmpv = pd.DataFrame()
dfnmpv['var'] = c1
dfnmpv['pv'] = nmpv
# Shapiron test (the closer to 1 the closer to normal distribution)
shpv = []
for i in range(0, len(c1)):
    shpv.append((stats.shapiro(acpr[c1[i]])).pvalue)
dfshpv = pd.DataFrame()
dfshpv['var'] = c1
dfshpv['pv'] = shpv
# applying yeojohnson transformation
acp_yj = pd.DataFrame()
acp_yj[c4[0]] = acpr[c4[0]]
for i in range(1, 35):
    acp_yj[c4[i]], lmbda = stats.yeojohnson(acpr[c4[i]])
# using scikit-learn powertransformer 
acp_yj1 = pd.DataFrame()
acp_yj1[c4[0]] = acpr[c4[0]]
for i in range(1, 35):
    ar = np.array(acpr[c4[i]]).reshape(-1,1)
    acp_yj1[c4[i]] = PowerTransformer(method = 'yeo-johnson', standardize = True).fit_transform(ar)

In [1]:
# checking normality
# normaltest pvalue (if it's bigger than alpha for example 0.05, we could say it's normal)
# nmpv_yj1 = []
# for i in range(0, len(c1)):
#     nmpv_yj1.append((stats.normaltest(acp_yj1[c1[i]])).pvalue)
# dfnmpv_yj1 = pd.DataFrame()
# dfnmpv_yj1['var'] = c1
# dfnmpv_yj1['pv'] = nmpv_yj1
# dfnmpv_yj1[dfnmpv_yj1['pv']>0.05]
# # Shapiron test (the closer to 1 the closer to normal distribution)
# shpv_yj1 = []
# for i in range(0, len(c1)):
#     shpv_yj1.append((stats.shapiro(acp_yj1[c1[i]])).pvalue)
# dfshpv_yj1 = pd.DataFrame()
# dfshpv_yj1['var'] = c1
# dfshpv_yj1['pv'] = shpv_yj1
# dfshpv_yj1[dfshpv_yj1['pv']>0.05]
# After checking, using yeojohnson from scikit-learn is better, since it's zero-mean, unit-variance normalization.

In [ ]:
# standardization
# Calculate zscore
acp_yjz1 = pd.DataFrame()
acp_yjz1[c4[0]] = acpr[c4[0]]
for i in range(1, 35):
    acp_yjz1[c4[i]] = stats.zscore(acp_yj1[c4[i]])
# Use minmax
sc = MinMaxScaler(feature_range = (-1,1))
# Transform into [-1,1]
acp_yj1z = pd.DataFrame()
acp_yj1z[c4[0]] = acpr[c4[0]]
for i in range(1, 35):
    ar = np.array(acp_yj1[c4[i]]).reshape(-1,1)
    acp_yj1z[c4[i]] = sc.fit_transform(ar)

In [2]:
# For categorical data transformation

In [206]:
# One hot encoding
acp_cat = acpr[c5].copy()
oh = ce.OneHotEncoder(cols = c2)
acp_catoh = oh.fit_transform(acp_cat)
# dropping useless columns
acp_catoh = acp_catoh.drop(columns = ['z_qnr_34', 'z_knr_12', 'trafficarea_3', 
                                      'r_width_7','speedlimit_7'])

In [4]:
# merging numeric and categorical variables
df_acp_transformed = acp_yj1z.merge(acp_catoh, on = ['Nr'])
# merging depedent variables
df_acpdv = acpr[['Nr','acp']].copy()
df_acp_transformed = df_acp_transformed.merge(df_acpdv, on = ['Nr'])
# saving to file
df_acp_transformed.to_csv('/home/yelhe/script/mtp/redo_model_input/model_acp_transformed_corr.csv', index = False)